<a href="https://colab.research.google.com/github/nxxk23/PROJECT/blob/main/FileVital.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
hemo = pd.read_csv('/content/drive/MyDrive/ckdresearch2023/Hemodialysis.csv', encoding='latin1')

In [3]:
import pandas as pd
vital = pd.read_csv('/content/drive/MyDrive/ckdresearch2023/vitalsign.csv')
vital.shape

(9213497, 7)

In [4]:
# Create a list from hemo['PatientUID']
patient_uid_list = hemo['PatientUID'].unique().tolist()

# Remove patient UIDs from visit['PatientUID'] that are not in the list
vital = vital[~vital['PatientUID'].isin(patient_uid_list)]
vital.shape

(8099274, 7)

In [5]:
# Specify the columns for unique value counting
columns_to_count = ['OrderItemName', 'ResultItemName']

# Count unique values in the specified columns
unique_counts = vital[columns_to_count].value_counts().reset_index()

In [6]:
unique_counts.columns = ['OrderItemName','ResultItemName', 'Count']

In [7]:
selected_items = ['Systolic BP', 'Diastolic BP', 'BMI', 'BSA']
filtered_items = unique_counts[unique_counts['ResultItemName'].isin(selected_items)]
filtered_items

,OrderItemName,ResultItemName,Count
0,Vital Signs Record,Systolic BP,1221173
1,Vital Signs Record,Diastolic BP,1220095
7,Vital Signs Record,BMI,558791
8,Vital Signs Record,BSA,558502


In [8]:
#filtered_items.to_excel('VitalFile.xlsx')

## **Slope**

> **ค่า x = ค่าจากวิสิท t ที่มาก่อน**




In [9]:
import pandas as pd
slope = pd.read_excel('/content/drive/MyDrive/ckdresearch2023/Assign/SummaryFile/Slope.xlsx')

In [10]:
slope

,Unnamed: 0,PatientUID,PatientVisitUID,CWhen,ResultItemName,ResultValue,date,datetime,TimeDiff,Slopes
0,0,2299,3386255,2559-08-15 09:51:54.913,eGFR(CKD-EPI) age >=18,71.70,2016-08-15,2016-08-15 09:51:54.913,NaN,NaN
1,4,2299,4546471,2560-10-03 09:19:54.710,eGFR(CKD-EPI) age >=18,63.60,2017-10-03,2017-10-03 09:19:54.710,1.131507,-7.158596
2,10,2299,5812902,2561-12-12 09:18:49.913,eGFR(CKD-EPI) age >=18,70.20,2018-12-12,2018-12-12 09:18:49.913,1.189041,5.550691
3,15,2299,6947220,2562-12-19 09:51:24.427,eGFR(CKD-EPI) age >=18,64.02,2019-12-19,2019-12-19 09:51:24.427,1.019178,-6.063710
4,23,2299,8186767,2564-03-04 10:01:42.113,eGFR(CKD-EPI) age >=18,48.54,2021-03-04,2021-03-04 10:01:42.113,1.208219,-12.812245
...,...,...,...,...,...,...,...,...,...,...
73587,318672,3421578,10151787,2565-12-14 11:30:53.310,eGFR(CKD-EPI) age >=18,47.44,2022-12-14,2022-12-14 11:30:53.310,NaN,NaN
73588,318674,3426284,10105952,2565-11-29 10:47:37.917,eGFR(CKD-EPI) age >=18,27.88,2022-11-29,2022-11-29 10:47:37.917,NaN,NaN
73589,318676,3431501,10144649,2565-12-13 13:42:48.510,eGFR(CKD-EPI) age >=18,54.65,2022-12-13,2022-12-13 13:42:48.510,NaN,NaN
73590,318678,3432056,10154545,2565-12-15 09:48:31.117,eGFR(CKD-EPI) age >=18,70.18,2022-12-15,2022-12-15 09:48:31.117,NaN,NaN


In [11]:
data = pd.DataFrame(slope)
data = data[['PatientUID','PatientVisitUID','Slopes']]
data = data.dropna()
data

,PatientUID,PatientVisitUID,Slopes
1,2299,4546471,-7.158596
2,2299,5812902,5.550691
3,2299,6947220,-6.063710
4,2299,8186767,-12.812245
5,2299,9385133,28.287500
...,...,...,...
73379,3319961,10553903,-4.256753
73381,3319971,10553916,-0.692078
73383,3320048,10572126,7.217875
73385,3320050,10572565,-32.794557


In [12]:
df = pd.DataFrame(data)

# Sort the DataFrame by PatientUID and PatientVisitUID
df = df.sort_values(by=['PatientUID', 'PatientVisitUID'])

# Create a new column to store the previous slope value
df['PreviousSlope'] = df['Slopes'].shift(1)

# Update the Slopes column with the PreviousSlope values
df['Slopes'] = df['PreviousSlope']

# Drop the PreviousSlope column
df = df.drop(columns=['PreviousSlope'])

In [13]:
data = df.dropna()

In [14]:
data

,PatientUID,PatientVisitUID,Slopes
2,2299,5812902,-7.158596
3,2299,6947220,5.550691
4,2299,8186767,-6.063710
5,2299,9385133,-12.812245
7,2304,4916314,28.287500
...,...,...,...
73379,3319961,10553903,10.568365
73381,3319971,10553916,-4.256753
73383,3320048,10572126,-0.692078
73385,3320050,10572565,7.217875


In [15]:
vital['ResultItemName'].nunique()

18

# **TestWideTable**


In [16]:

# Initialize a dictionary to store the vital values
vital_dict = {}

# Iterate through unique ResultItemName values
for result_item in vital['ResultItemName'].unique():
    # Filter vital DataFrame for the current result_item
    subset_vital = vital[vital['ResultItemName'] == result_item]

    # Merge the subset_vital with data on PatientUID and PatientVisitUID
    merged_data = pd.merge(data, subset_vital[['PatientUID', 'PatientVisitUID', 'ResultValue', 'CWhen']], on=['PatientUID', 'PatientVisitUID'], how='left')

    # Rename the ResultValue column to match the result_item
    merged_data.rename(columns={'ResultValue': result_item}, inplace=True)

    # Update the vital_dict with the result_item as the key and merged_data as the value
    vital_dict[result_item] = merged_data

# Concatenate the dictionary values to get the final result
final_result = pd.concat(vital_dict.values(), axis=1)

# Drop duplicate columns (except for 'PatientUID', 'PatientVisitUID', and 'Slopes')
final_result = final_result.loc[:, ~final_result.columns.duplicated()]


# **solved the dup rows**


In [17]:

# Sort data by 'CWhen' in descending order within each patient's visit and keep the latest visit row
final_result.sort_values(by=['PatientUID', 'PatientVisitUID', 'CWhen'], ascending=[True, True, False], inplace=True)
final_result.drop_duplicates(subset=['PatientUID', 'PatientVisitUID'], keep='first', inplace=True)


In [18]:
vitalDF = final_result[['PatientUID','PatientVisitUID','Slopes','Systolic BP', 'Diastolic BP', 'BMI', 'BSA']]
# Drop rows with missing values in 'PatientUID', 'PatientVisitUID', and 'Slopes'
vitalDF = vitalDF.dropna(subset=['PatientUID', 'PatientVisitUID', 'Slopes'])

In [19]:
vitalDF.shape

(52619, 7)

In [20]:
data.shape

(52619, 3)

In [21]:
vitalDF

,PatientUID,PatientVisitUID,Slopes,Systolic BP,Diastolic BP,BMI,BSA
0,2299.0,5812902.0,-7.158596,138,76,20.72,1.38
1,2299.0,6947220.0,5.550691,134,66,20.27,1.36
3,2299.0,8186767.0,-6.063710,103,57,19.05,1.33
4,2299.0,9385133.0,-12.812245,128,62,NaN,NaN
5,2304.0,4916314.0,28.287500,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
54064,3319961.0,10553903.0,10.568365,147,87,NaN,NaN
54065,3319971.0,10553916.0,-4.256753,143,86,NaN,NaN
54066,3320048.0,10572126.0,-0.692078,131,79,NaN,NaN
54067,3320050.0,10572565.0,7.217875,138,NaN,NaN,NaN


## **Check Duplicate vitalDF**
    cuz there are the same visitID but in different Lab
    

In [22]:
# Check for duplicate rows in data
duplicates_data = data[data.duplicated(['PatientUID', 'PatientVisitUID'], keep=False)]

# Check for duplicate rows in vitalDF
duplicates_vitalDF = vitalDF[vitalDF.duplicated(['PatientUID', 'PatientVisitUID', 'Slopes'], keep=False)]


In [23]:
duplicates_vitalDF

,PatientUID,PatientVisitUID,Slopes,Systolic BP,Diastolic BP,BMI,BSA


    this is visitID that had dup value
    but we select only latest value

In [24]:
vital[vital['PatientVisitUID']==4286612]

,LabNo,PatientUID,PatientVisitUID,OrderItemName,ResultItemName,ResultValue,CWhen
338602,1,2299,4286612,Vital Signs Record,BSA,1.42,2560-07-05 09:00:18.093
338603,2,2299,4286612,Vital Signs Record,BSA,1.41,2560-07-05 14:29:33.277
338604,1,2299,4286612,Vital Signs Record,Systolic BP,138,2560-07-05 09:00:18.093
338605,2,2299,4286612,Vital Signs Record,Systolic BP,129,2560-07-05 14:29:33.277
338606,1,2299,4286612,Vital Signs Record,Waistline,72,2560-07-05 14:29:33.277
338607,1,2299,4286612,Vital Signs Record,Weight,48.6,2560-07-05 09:00:18.093
338608,2,2299,4286612,Vital Signs Record,Weight,48.7,2560-07-05 14:29:33.277
338753,1,2299,4286612,Vital Signs Record,BMI,21.6,2560-07-05 09:00:18.093
338754,2,2299,4286612,Vital Signs Record,BMI,21.94,2560-07-05 14:29:33.277
338755,1,2299,4286612,Vital Signs Record,Pulse Rate,76,2560-07-05 09:00:18.093


In [25]:
vitalDF[vitalDF['PatientVisitUID']==4286612.0]

,PatientUID,PatientVisitUID,Slopes,Systolic BP,Diastolic BP,BMI,BSA


In [26]:
vitalDF

,PatientUID,PatientVisitUID,Slopes,Systolic BP,Diastolic BP,BMI,BSA
0,2299.0,5812902.0,-7.158596,138,76,20.72,1.38
1,2299.0,6947220.0,5.550691,134,66,20.27,1.36
3,2299.0,8186767.0,-6.063710,103,57,19.05,1.33
4,2299.0,9385133.0,-12.812245,128,62,NaN,NaN
5,2304.0,4916314.0,28.287500,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
54064,3319961.0,10553903.0,10.568365,147,87,NaN,NaN
54065,3319971.0,10553916.0,-4.256753,143,86,NaN,NaN
54066,3320048.0,10572126.0,-0.692078,131,79,NaN,NaN
54067,3320050.0,10572565.0,7.217875,138,NaN,NaN,NaN


In [28]:
vitalDF.to_excel('VitalDF.xlsx')